# RGB resnet test

In [1]:
import torch.nn as nn
import torch
import math
import torch.utils.model_zoo as model_zoo

In [2]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [3]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


In [10]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        # self.fc_aux = nn.Linear(512 * block.expansion, 101)
        self.dp = nn.Dropout(p=0.8)
        self.fc_action = nn.Linear(512 * block.expansion, num_classes)
        # self.bn_final = nn.BatchNorm1d(num_classes)
        # self.fc2 = nn.Linear(num_classes, num_classes)
        # self.fc_final = nn.Linear(num_classes, 101)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dp(x)
        x = self.fc_action(x)
        # x = self.bn_final(x)
        # x = self.fc2(x)
        # x = self.fc_final(x)

        return x


def rgb_resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def rgb_resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
#     model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    if pretrained:
        pretrained_dict = model_zoo.load_url(model_urls['resnet34'])
        print("pretrained_dict's state_dict:", len(pretrained_dict))
        for i, param_tensor in enumerate(pretrained_dict):
            print(i, param_tensor, "\t", pretrained_dict[param_tensor].size())
            
    
        model_dict = model.state_dict()
        print('------'*20)
        print("model_dict's state_dict:", len(model_dict))
        for i, param_tensor in enumerate(model_dict):
            print(i, param_tensor, "\t", model_dict[param_tensor].size())
            
            
        # 1. filter out unnecessary keys
        # 沒有在model_dict裡面的key 就拿掉
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        
        print('------'*20)
        print("filter out unnecessary keys pretrained_dict's state_dict:", len(pretrained_dict))
        for i, param_tensor in enumerate(pretrained_dict):
            print(i, param_tensor, "\t", pretrained_dict[param_tensor].size())
            
        # 2. overwrite entries in the existing state dict
        # use pretrained_dict value , update model_dict value
        model_dict.update(pretrained_dict) 
        print('------'*20)
        print("update model_dict's state_dict:", len(model_dict))
        for i, param_tensor in enumerate(model_dict):
            print(i, param_tensor, "\t", model_dict[param_tensor].size())
                        
        # 3. load the new state dict
        model.load_state_dict(model_dict)
        
    return model

# 50 以後用Bottleneck
def rgb_resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        # model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
        pretrained_dict = model_zoo.load_url(model_urls['resnet50'])

        model_dict = model.state_dict()

        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        # 3. load the new state dict
        model.load_state_dict(model_dict)

    return model


def rgb_resnet50_aux(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        # model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
        pretrained_dict = model_zoo.load_url(model_urls['resnet50'])

        model_dict = model.state_dict()
        fc_origin_weight = pretrained_dict["fc.weight"].data.numpy()
        fc_origin_bias = pretrained_dict["fc.bias"].data.numpy()

        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        # print(model_dict)
        fc_new_weight = model_dict["fc_aux.weight"].numpy() 
        fc_new_bias = model_dict["fc_aux.bias"].numpy() 

        fc_new_weight[:1000, :] = fc_origin_weight
        fc_new_bias[:1000] = fc_origin_bias

        model_dict["fc_aux.weight"] = torch.from_numpy(fc_new_weight)
        model_dict["fc_aux.bias"] = torch.from_numpy(fc_new_bias)

        # 3. load the new state dict
        model.load_state_dict(model_dict)

    return model


def rgb_resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def rgb_resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        # model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
        pretrained_dict = model_zoo.load_url(model_urls['resnet152'])
        model_dict = model.state_dict()

        # 1. filter out unnecessary keys
        # 沒有在model_dict裡面的key 就拿掉
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # 2. overwrite entries in the existing state dict
        # use pretrained_dict value , update model_dict value
        model_dict.update(pretrained_dict) 
        # 3. load the new state dict
        model.load_state_dict(model_dict)

    return model

In [11]:
model = rgb_resnet34(True, num_classes=2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\leech/.cache\torch\hub\checkpoints\resnet34-333f7ec4.pth



pretrained_dict's state_dict: 182
0 conv1.weight 	 torch.Size([64, 3, 7, 7])
1 bn1.running_mean 	 torch.Size([64])
2 bn1.running_var 	 torch.Size([64])
3 bn1.weight 	 torch.Size([64])
4 bn1.bias 	 torch.Size([64])
5 layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
6 layer1.0.bn1.running_mean 	 torch.Size([64])
7 layer1.0.bn1.running_var 	 torch.Size([64])
8 layer1.0.bn1.weight 	 torch.Size([64])
9 layer1.0.bn1.bias 	 torch.Size([64])
10 layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
11 layer1.0.bn2.running_mean 	 torch.Size([64])
12 layer1.0.bn2.running_var 	 torch.Size([64])
13 layer1.0.bn2.weight 	 torch.Size([64])
14 layer1.0.bn2.bias 	 torch.Size([64])
15 layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
16 layer1.1.bn1.running_mean 	 torch.Size([64])
17 layer1.1.bn1.running_var 	 torch.Size([64])
18 layer1.1.bn1.weight 	 torch.Size([64])
19 layer1.1.bn1.bias 	 torch.Size([64])
20 layer1.1.conv2.weight 	 torch.Size([64, 64, 3, 3])
21 layer1.1.bn2.running_mean 	 torch.Size([

14 layer1.0.bn2.bias 	 torch.Size([64])
15 layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
16 layer1.1.bn1.running_mean 	 torch.Size([64])
17 layer1.1.bn1.running_var 	 torch.Size([64])
18 layer1.1.bn1.weight 	 torch.Size([64])
19 layer1.1.bn1.bias 	 torch.Size([64])
20 layer1.1.conv2.weight 	 torch.Size([64, 64, 3, 3])
21 layer1.1.bn2.running_mean 	 torch.Size([64])
22 layer1.1.bn2.running_var 	 torch.Size([64])
23 layer1.1.bn2.weight 	 torch.Size([64])
24 layer1.1.bn2.bias 	 torch.Size([64])
25 layer1.2.conv1.weight 	 torch.Size([64, 64, 3, 3])
26 layer1.2.bn1.running_mean 	 torch.Size([64])
27 layer1.2.bn1.running_var 	 torch.Size([64])
28 layer1.2.bn1.weight 	 torch.Size([64])
29 layer1.2.bn1.bias 	 torch.Size([64])
30 layer1.2.conv2.weight 	 torch.Size([64, 64, 3, 3])
31 layer1.2.bn2.running_mean 	 torch.Size([64])
32 layer1.2.bn2.running_var 	 torch.Size([64])
33 layer1.2.bn2.weight 	 torch.Size([64])
34 layer1.2.bn2.bias 	 torch.Size([64])
35 layer2.0.conv1.weight 	 torch.Siz

206 layer4.2.bn1.bias 	 torch.Size([512])
207 layer4.2.bn1.running_mean 	 torch.Size([512])
208 layer4.2.bn1.running_var 	 torch.Size([512])
209 layer4.2.bn1.num_batches_tracked 	 torch.Size([])
210 layer4.2.conv2.weight 	 torch.Size([512, 512, 3, 3])
211 layer4.2.bn2.weight 	 torch.Size([512])
212 layer4.2.bn2.bias 	 torch.Size([512])
213 layer4.2.bn2.running_mean 	 torch.Size([512])
214 layer4.2.bn2.running_var 	 torch.Size([512])
215 layer4.2.bn2.num_batches_tracked 	 torch.Size([])
216 fc_action.weight 	 torch.Size([2, 512])
217 fc_action.bias 	 torch.Size([2])


In [16]:
my_dict = {'bill':3, 'rich':10}
yourDict = {'bill':21, 'aa':20}
my_dict.update(yourDict)
print(my_dict)

{'bill': 21, 'rich': 10, 'aa': 20}


In [21]:
import collections
d4=collections.OrderedDict()
d4['a']='A'
d4['b']='B'
d4['c']='C'

d5=collections.OrderedDict()
d5['c1']='C'
d5['a1']='A'
d5['b1']='B'
d4.update(d5)
print(d4)

OrderedDict([('a', 'A'), ('b', 'B'), ('c', 'C'), ('c1', 'C'), ('a1', 'A'), ('b1', 'B')])


In [24]:
allKeyList = d4.keys()
print(allKeyList)

odict_keys(['a', 'b', 'c', 'c1', 'a1', 'b1'])


In [25]:
lst = list(allKeyList)
print(lst)

['a', 'b', 'c', 'c1', 'a1', 'b1']


In [27]:
layer_key = 'layer1.0.conv1.weight'
lay= layer_key.split('.', 1)
new_layer_name = '_a.'.join(lay)
print(lay, new_layer_name)

['layer1', '0.conv1.weight'] layer1_a.0.conv1.weight
